In [2]:

import torch
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as plt
import sklearn
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import scipy.io as sio
import numpy
import numpy as np
import scipy.io as sci
import torch.nn as nn
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms as T
from torchvision.transforms import functional as F
import torch as T
from torchvision import datasets
import torch.utils as utils
from sklearn.metrics import confusion_matrix
import joblib

In [ ]:
pip install seaborn

In [ ]:
pip install torchvision

In [ ]:
pip install --upgrade pip

KMNIST references:
* [GitHub page](https://github.com/rois-codh/kmnist)
* [research paper](https://arxiv.org/pdf/1812.01718.pdf)

EMNIST references:
* [NIST page](https://www.nist.gov/itl/products-and-services/emnist-dataset)
* [research paper](https://arxiv.org/pdf/1702.05373v1.pdf)

In [3]:
from os import listdir as ls
ls("/home/DAVIDSON/brwiedenbeck/public")

['kmnist', 'NLP', 'chest_xrays', 'brain_scans', 'birds', 'emnist']

In [4]:
#loads device 
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [5]:
#load data
mat = sci.loadmat("/home/DAVIDSON/brwiedenbeck/public/emnist/emnist-balanced.mat")


data = mat['dataset']
x_train = data['train'][0,0]['images'][0,0]/255
y_train = data['train'][0,0]['labels'][0,0]
x_test =  data['test'][0,0]['images'][0,0]/255
y_test =  data['test'][0,0]['labels'][0,0]

In [6]:
#convert to tensor
x_Train = T.tensor(x_train,
      dtype=T.float32).to(device)
y_Train = T.tensor(y_train,
      dtype=T.long).to(device)
x_Test = T.tensor(x_test,
      dtype=T.float32).to(device)
y_Test = T.tensor(y_test,
      dtype=T.long).to(device)

In [7]:
#Reshape to 1D
y_Train = y_Train.reshape([112800])
y_Test = y_Test.reshape([18800])

In [8]:
y_Train.shape

torch.Size([112800])

In [9]:
#one_hot_encoding
y_Train = torch.nn.functional.one_hot(y_Train)
y_Test = torch.nn.functional.one_hot(y_Test)

In [10]:
y_Train.shape

torch.Size([112800, 47])

In [11]:
#combine train label and image dataset
train_dataset = utils.data.TensorDataset(x_Train, y_Train)
test_dataset = utils.data.TensorDataset(x_Test, y_Test)

In [12]:
#split train and validation data set
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset_dl, val_dataset_dl = torch.utils.data.random_split(train_dataset, [train_size, val_size])
print(f'train set size: {train_dataset_dl}, validation set size: {val_dataset_dl}')

train set size: <torch.utils.data.dataset.Subset object at 0x153b668d4670>, validation set size: <torch.utils.data.dataset.Subset object at 0x153b666c34f0>


In [13]:
# Create data loaders
batch_size = 5
train_dataset_b = torch.utils.data.DataLoader(train_dataset_dl, batch_size=batch_size, shuffle=True)
test_dataset_b = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
val_dataset_b = torch.utils.data.DataLoader(val_dataset_dl, batch_size=batch_size)

In [67]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 800),
            nn.Linear(800, 800),
            nn.Linear(800, 800),
            nn.Linear(800, 800),
            nn.Linear(800, 512),
            nn.Dropout(p=0.2),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.Dropout(p=0.2),
            nn.Linear(512, 47)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=800, bias=True)
    (3): Linear(in_features=800, out_features=800, bias=True)
    (4): Linear(in_features=800, out_features=800, bias=True)
    (5): Linear(in_features=800, out_features=800, bias=True)
    (6): Linear(in_features=800, out_features=512, bias=True)
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=512, out_features=512, bias=True)
    (9): ReLU()
    (10): Linear(in_features=512, out_features=512, bias=True)
    (11): Dropout(p=0.2, inplace=False)
    (12): Linear(in_features=512, out_features=47, bias=True)
  )
)


In [68]:
#Optimizing the model
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [69]:
train_losses = []
val_losses = []
def train(train_dataset_b, model, loss_fn, optimizer):
    size = len(train_dataset_b.dataset)
    model.train()
    for i, (images, labels) in enumerate(train_dataset_b):
        # Compute prediction error
        pred = model(images)
        loss = loss_fn(pred, labels.float())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_size % 100 == 0:
            loss, current = loss.item(), batch * len(images)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]") 
    
    valid_loss = 0.0 
    model.eval()     
    for images, labels in val_dataset_b:
        if torch.cuda.is_available():
            images, labels = images.cuda(), labels.cuda()
        pred = model(images)
        loss = loss_fn(pred, labels.float())
        valid_loss = loss.item()
    print(f'\t\t Training Loss: {loss/len(train_dataset_b.dataset)} \t\t Validation Loss: {valid_loss / len(val_dataset_b.dataset)}')
    train_loss_values = loss/len(train_dataset_b.dataset)
    train_loss = train_loss_values.item()
    train_losses.append(train_loss)   
    val_losses.append(valid_loss/len(val_dataset_b.dataset))

In [70]:
train(train_dataset_b, model, loss_fn, optimizer)

		 Training Loss: 2.2665075448458083e-05 		 Validation Loss: 9.066030277428052e-05


In [71]:
test_losses = []
test_accuracy=[]
def test(test_dataset_b, model, loss_fn):
    size = len(test_dataset_b.dataset)
    num_batches = len(test_dataset_b)
     
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for i, (images, labels) in enumerate (test_dataset_b):
            images, labels = images.to(device), labels.to(device)
            pred = model(images)
            test_loss += loss_fn(pred, labels.float()).item()
            correct += (pred.argmax(1) == labels.argmax(1)).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    test_accuracy.append(100*correct)
    test_losses.append(test_loss)

In [72]:
test(test_dataset_b, model, loss_fn)

Test Error: 
 Accuracy: 56.9%, Avg loss: 1.358505 



In [73]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataset_b, model, loss_fn, optimizer)
    test(test_dataset_b, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
		 Training Loss: 7.3462365435261745e-06 		 Validation Loss: 2.9384947203575298e-05
Test Error: 
 Accuracy: 73.9%, Avg loss: 0.794009 

Epoch 2
-------------------------------
		 Training Loss: 2.8990316423005424e-06 		 Validation Loss: 1.1596127234874887e-05
Test Error: 
 Accuracy: 77.2%, Avg loss: 0.696051 

Epoch 3
-------------------------------
		 Training Loss: 2.9321490728762e-06 		 Validation Loss: 1.172859697265828e-05
Test Error: 
 Accuracy: 77.7%, Avg loss: 0.651907 

Epoch 4
-------------------------------
		 Training Loss: 2.8321264835540205e-06 		 Validation Loss: 1.1328506078703184e-05
Test Error: 
 Accuracy: 79.8%, Avg loss: 0.628143 

Epoch 5
-------------------------------
		 Training Loss: 5.116264219395816e-06 		 Validation Loss: 2.0465056629891092e-05
Test Error: 
 Accuracy: 81.0%, Avg loss: 0.595192 

Epoch 6
-------------------------------
		 Training Loss: 3.572298055587453e-06 		 Validation Loss: 1.4289192432630147e-05
Te